# Import and Read data


In [1]:
import sys
sys.path.append('.')  # Add current directory to path
from build_graph_and_train import *

/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
partition = 300

In [3]:
df = pd.read_csv(f"../../../../data/top30groups/DNDF_OneHotLongLatCombined/combined/combined{partition}.csv")

In [4]:
import os 
if not os.path.isdir(f"Results{partition}"):
    os.mkdir(f"Results{partition}")

# Create longlat feature

In [5]:
geodata = ['longitude', 'latitude']
combined_geo = df.copy()
combined_geo['longlat'] = list(zip(df['longitude'], df['latitude']))
combined_geo = combined_geo.drop(columns=geodata)

In [6]:
import ast

def to_tuple_if_needed(val):
    if isinstance(val, str):
        return ast.literal_eval(val)
    return val  # already a tuple

combined_geo['longlat'] = combined_geo['longlat'].apply(to_tuple_if_needed)

In [7]:
combined_geo.dtypes

extended         int64
vicinity         int64
multiple       float64
success          int64
suicide          int64
attacktype1      int64
targtype1        int64
target1          int64
individual       int64
weaptype1        int64
nkill          float64
property         int64
ishostkid      float64
gname           object
longlat         object
dtype: object

# Weapon type prediction

In [8]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'weaptype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 201.0163 | NRF Loss: 3.3982 | JOINT Loss: 204.4145 | NRF Acc: 0.0528
Epoch 02 | GCN MSE Loss: 147.6738 | NRF Loss: 3.3930 | JOINT Loss: 151.0668 | NRF Acc: 0.0556
Epoch 03 | GCN MSE Loss: 102.7943 | NRF Loss: 3.3874 | JOINT Loss: 106.1817 | NRF Acc: 0.0412
Epoch 04 | GCN MSE Loss: 69.3109 | NRF Loss: 3.3850 | JOINT Loss: 72.6959 | NRF Acc: 0.0428
Epoch 05 | GCN MSE Loss: 45.3611 | NRF Loss: 3.3904 | JOINT Loss: 48.7516 | NRF Acc: 0.0403
Epoch 06 | GCN MSE Loss: 29.8861 | NRF Loss: 3.3911 | JOINT Loss: 33.2772 | NRF Acc: 0.0369
Epoch 07 | GCN MSE Loss: 22.1011 | NRF Loss: 3.3935 | JOINT Loss: 25.4946 | NRF Acc: 0.0325
Epoch 08 | GCN MSE Loss: 20.3761 | NRF Loss: 3.3944 | JOINT Loss: 23.7704 | NRF Acc: 0.0239
Epoch 09 | GCN MSE Loss: 22.1887 | NRF Loss: 3.3975 | JOINT Loss: 25.5862 | NRF Acc: 0.0197
Epoch 10 | GCN MSE Loss: 25.4273 | NRF Loss: 3.3956 | JOINT Loss: 28.8229 | NRF Acc: 0.0185
Epoch 11 | GCN MSE Loss: 28.7546 | NRF Loss: 3.3952 | JOINT Loss: 32.1499 

# Fatality rate


In [9]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'nkill'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 824.5430 | NRF Loss: 3.4037 | JOINT Loss: 827.9467 | NRF Acc: 0.0545
Epoch 02 | GCN MSE Loss: 653.1412 | NRF Loss: 3.3952 | JOINT Loss: 656.5364 | NRF Acc: 0.0544
Epoch 03 | GCN MSE Loss: 510.3559 | NRF Loss: 3.3934 | JOINT Loss: 513.7493 | NRF Acc: 0.0485
Epoch 04 | GCN MSE Loss: 394.6765 | NRF Loss: 3.3915 | JOINT Loss: 398.0680 | NRF Acc: 0.0209
Epoch 05 | GCN MSE Loss: 304.1343 | NRF Loss: 3.3878 | JOINT Loss: 307.5221 | NRF Acc: 0.0209
Epoch 06 | GCN MSE Loss: 236.4070 | NRF Loss: 3.3912 | JOINT Loss: 239.7983 | NRF Acc: 0.0209
Epoch 07 | GCN MSE Loss: 189.1707 | NRF Loss: 3.3930 | JOINT Loss: 192.5637 | NRF Acc: 0.0209
Epoch 08 | GCN MSE Loss: 159.7316 | NRF Loss: 3.3958 | JOINT Loss: 163.1274 | NRF Acc: 0.0209
Epoch 09 | GCN MSE Loss: 145.0548 | NRF Loss: 3.3968 | JOINT Loss: 148.4515 | NRF Acc: 0.0209
Epoch 10 | GCN MSE Loss: 142.0153 | NRF Loss: 3.3962 | JOINT Loss: 145.4115 | NRF Acc: 0.0209
Epoch 11 | GCN MSE Loss: 146.9730 | NRF Loss: 3.3954 | JOINT

# Target Type

In [10]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'targtype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 470.4682 | NRF Loss: 3.3984 | JOINT Loss: 473.8666 | NRF Acc: 0.0532
Epoch 02 | GCN MSE Loss: 346.1680 | NRF Loss: 3.3864 | JOINT Loss: 349.5544 | NRF Acc: 0.0556
Epoch 03 | GCN MSE Loss: 248.0323 | NRF Loss: 3.3864 | JOINT Loss: 251.4187 | NRF Acc: 0.0571
Epoch 04 | GCN MSE Loss: 172.4672 | NRF Loss: 3.3847 | JOINT Loss: 175.8519 | NRF Acc: 0.0578
Epoch 05 | GCN MSE Loss: 119.0717 | NRF Loss: 3.3843 | JOINT Loss: 122.4559 | NRF Acc: 0.0407
Epoch 06 | GCN MSE Loss: 86.2505 | NRF Loss: 3.3846 | JOINT Loss: 89.6351 | NRF Acc: 0.0391
Epoch 07 | GCN MSE Loss: 70.0794 | NRF Loss: 3.3860 | JOINT Loss: 73.4655 | NRF Acc: 0.0403
Epoch 08 | GCN MSE Loss: 66.2040 | NRF Loss: 3.3889 | JOINT Loss: 69.5929 | NRF Acc: 0.0426
Epoch 09 | GCN MSE Loss: 70.8778 | NRF Loss: 3.3860 | JOINT Loss: 74.2639 | NRF Acc: 0.0440
Epoch 10 | GCN MSE Loss: 80.5417 | NRF Loss: 3.3839 | JOINT Loss: 83.9257 | NRF Acc: 0.0467
Epoch 11 | GCN MSE Loss: 91.3974 | NRF Loss: 3.3849 | JOINT Loss: 94.7

# Attack type

In [11]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'attacktype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 537.7028 | NRF Loss: 3.3986 | JOINT Loss: 541.1014 | NRF Acc: 0.0537
Epoch 02 | GCN MSE Loss: 407.6705 | NRF Loss: 3.3951 | JOINT Loss: 411.0656 | NRF Acc: 0.0578
Epoch 03 | GCN MSE Loss: 300.6505 | NRF Loss: 3.3951 | JOINT Loss: 304.0456 | NRF Acc: 0.0424
Epoch 04 | GCN MSE Loss: 215.2281 | NRF Loss: 3.3959 | JOINT Loss: 218.6240 | NRF Acc: 0.0296
Epoch 05 | GCN MSE Loss: 149.8536 | NRF Loss: 3.3954 | JOINT Loss: 153.2490 | NRF Acc: 0.0232
Epoch 06 | GCN MSE Loss: 101.7474 | NRF Loss: 3.3940 | JOINT Loss: 105.1414 | NRF Acc: 0.0232
Epoch 07 | GCN MSE Loss: 68.0579 | NRF Loss: 3.3968 | JOINT Loss: 71.4548 | NRF Acc: 0.0229
Epoch 08 | GCN MSE Loss: 46.0971 | NRF Loss: 3.3983 | JOINT Loss: 49.4954 | NRF Acc: 0.0256
Epoch 09 | GCN MSE Loss: 33.3876 | NRF Loss: 3.3963 | JOINT Loss: 36.7839 | NRF Acc: 0.0274
Epoch 10 | GCN MSE Loss: 27.5829 | NRF Loss: 3.3933 | JOINT Loss: 30.9762 | NRF Acc: 0.0279
Epoch 11 | GCN MSE Loss: 26.5298 | NRF Loss: 3.3907 | JOINT Loss: 29